# Rock Paper Scissors Classification 

This notebook ensures consistent and correct class label ordering by explicitly sorting class folders alphabetically.

## 1. Install and Import Libraries

In [ ]:

!pip install datasets --quiet

import os
import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
from datasets import load_dataset
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


## 2. Extract Dataset from Hugging Face

In [ ]:

dataset = load_dataset("Javtor/rock-paper-scissors")
label_names = dataset['train'].features['label'].names

base_dir = "dataset"
splits = ['train', 'test']

def save_images_to_folder(split_name):
    split_dataset = dataset[split_name]
    for idx, sample in enumerate(split_dataset):
        label = label_names[sample['label']]
        image = sample['image']
        if image.mode == 'RGBA':
            image = image.convert('RGB')
        save_dir = os.path.join(base_dir, split_name, label)
        os.makedirs(save_dir, exist_ok=True)
        image_path = os.path.join(save_dir, f"{split_name}_{label}_{idx}.jpg")
        image.save(image_path)

for split in splits:
    save_images_to_folder(split)


## 3. Enforce Alphabetical Class Label Order

In [ ]:

train_dir = 'dataset/train'
class_names = sorted(os.listdir(train_dir))  # ['paper', 'rock', 'scissors']
print("✅ Sorted class names used for prediction:", class_names)

with open("class_names.json", "w") as f:
    json.dump(class_names, f)


## 4. Create Augmented Data Generators

In [ ]:

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.3,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    brightness_range=[0.7, 1.3],
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir, target_size=(224, 224), batch_size=32,
    class_mode='categorical', subset='training'
)
val_gen = train_datagen.flow_from_directory(
    train_dir, target_size=(224, 224), batch_size=32,
    class_mode='categorical', subset='validation'
)
test_gen = test_datagen.flow_from_directory(
    'dataset/test', target_size=(224, 224), batch_size=32, class_mode='categorical'
)


## 5. Compute Class Weights

In [ ]:

y_train = train_gen.classes
weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(weights))


## 6. Define and Compile Model

In [ ]:

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(train_gen.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

callbacks = [
    EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, min_lr=1e-6)
]

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='categorical_crossentropy', metrics=['accuracy'])


## 7. Train Model

In [ ]:

history = model.fit(train_gen, validation_data=val_gen,
                    epochs=10, class_weight=class_weights, callbacks=callbacks)


## 8. Fine-Tune Last 20 Layers

In [ ]:

base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy', metrics=['accuracy'])

fine_tune_history = model.fit(train_gen, validation_data=val_gen,
                              epochs=5, class_weight=class_weights, callbacks=callbacks)


## 9. Plot Training Results

In [ ]:

def smooth_curve(points, factor=0.8):
    smoothed = []
    for point in points:
        if smoothed:
            smoothed.append(smoothed[-1] * factor + point * (1 - factor))
        else:
            smoothed.append(point)
    return smoothed

def combine_history(h1, h2):
    combined = {}
    for k in h1.history:
        combined[k] = h1.history[k] + h2.history.get(k, [])
    return combined

full_history = combine_history(history, fine_tune_history)

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(smooth_curve(full_history['accuracy']), label='Training Acc')
plt.plot(smooth_curve(full_history['val_accuracy']), label='Val Acc')
plt.legend()
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(smooth_curve(full_history['loss']), label='Training Loss')
plt.plot(smooth_curve(full_history['val_loss']), label='Val Loss')
plt.legend()
plt.title('Loss')
plt.tight_layout()
plt.show()


## 10. Export Model

In [ ]:

model.save("rock_paper_scissors_model.h5")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("rock_paper_scissors_model.tflite", "wb") as f:
    f.write(tflite_model)
